In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
from random import randint


In [2]:
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')


In [3]:
#creating an empty list, so that we can append the values
movie_name = []
year = []
time=[]
rating=[]
metascore =[]
votes = []
gross = []
description = []

In [4]:
#storing the meaningfull required data in the variable
pages = np.arange(1,1000,100)

In [5]:
#calling one by one using for loop
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(year_of_release)
        
        runtime = store.p.find("span", class_ = 'runtime').text
        time.append(runtime)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else "NA"
        metascore.append(meta)
        
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 else 'NA'
        gross.append(grosses)

In [6]:
# Description of the Movies
describe = store.find_all('p', class_ = 'text-muted')
description_ = describe[1].text.replace('\n', '') if len(describe) >1 else 'NA'
description.append(description_)

In [7]:
#Creating a dataframe using pandas library
movie_list = { "Movie Name": movie_name, "Year of Release" : year, "Watch Time": time,"Movie Rating": rating, "Metascore of movie": metascore, "Votes" : votes, "Gross": gross, "Description": description  }
df = pd.DataFrame.from_dict(movie_list, orient='index')
df.transpose()

,Movie Name,Year of Release,Watch Time,Movie Rating,Metascore of movie,Votes,Gross,Description
0,Jai Bhim,(2021),164 min,9.5,NA,"152,430",NA,A man in London tries to help a counter-espion...
1,The Shawshank Redemption,(1994),142 min,9.3,80,"2,501,001",$28.34M,None
2,The Godfather,(1972),175 min,9.2,100,"1,724,313",$134.97M,None
3,Soorarai Pottru,(2020),153 min,9.1,NA,"103,930",NA,None
4,The Dark Knight,(2008),152 min,9.0,84,"2,450,916",$534.86M,None
...,...,...,...,...,...,...,...,...
995,Sabrina,(1954),113 min,7.6,72,"62,665",NA,None
996,From Here to Eternity,(1953),118 min,7.6,85,"45,600",$30.50M,None
997,Gilda,(1946),110 min,7.6,NA,"31,193",NA,None
998,Lifeboat,(1944),97 min,7.6,78,"28,029",NA,None
